In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
open("/Users/olivianystrom/Desktop/Springboard/CA_Birds_of_Prey/Birds_of_Prey.csv")
df = pd.read_csv("/Users/olivianystrom/Desktop/Springboard/CA_Birds_of_Prey/Birds_of_Prey.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
print(df.head())
print(df['Size'].unique())
print(df.columns)
print(df['COMMON NAME'].value_counts())

   Unnamed: 0.1            LAST EDITED DATE        COMMON NAME  \
0             0  2021-08-05 02:45:57.629774         Bald Eagle   
1             1  2021-04-01 02:35:26.368505      Cooper's Hawk   
2             2  2021-04-01 02:35:26.368505    Red-tailed Hawk   
3             3         2017-08-16 00:55:22   American Kestrel   
4             4         2014-10-19 02:09:50  White-tailed Kite   

            SCIENTIFIC NAME     COUNTY COUNTY CODE   LATITUDE   LONGITUDE  \
0  Haliaeetus leucocephalus  Del Norte   US-CA-015  41.754666 -124.202714   
1        Accipiter cooperii  Riverside   US-CA-065  33.531253 -117.075205   
2         Buteo jamaicensis  Riverside   US-CA-065  33.462797 -115.980082   
3          Falco sparverius       Kern   US-CA-029  35.615712 -118.251686   
4           Elanus leucurus   Monterey   US-CA-053  36.826080 -121.632273   

  OBSERVATION DATE                    Size  ... Orange  Black  \
0       1999-01-18   goose-sized or larger  ...      0      0   
1       19

In [4]:
# removing valuew with incidence under 1000 sightings
low_incidence = ["Harris's Hawk", 'Gray Hawk', 'Common Black Hawk', 'Mississippi Kite', 
                 'Snowy Owl', 'Swallow-tailed Kite', 'Elf Owl', 'Aplomado Falcon'] # Removing non-raptor species that remained in the dataset
df.drop(df[df['COMMON NAME'].str.contains('|'.join(low_incidence))].index, inplace = True)
print(df['COMMON NAME'].value_counts())


Red-tailed Hawk          1176986
Red-shouldered Hawk       574831
American Kestrel          528448
Cooper's Hawk             370733
Northern Harrier          334597
White-tailed Kite         238140
Peregrine Falcon          165279
Great Horned Owl          146301
Sharp-shinned Hawk        102859
Bald Eagle                 92167
Merlin                     78960
Barn Owl                   67592
Golden Eagle               64187
Swainson's Hawk            64171
Burrowing Owl              50720
Ferruginous Hawk           48514
Prairie Falcon             37034
Western Screech-Owl        29922
Rough-legged Hawk          14697
Northern Pygmy-Owl         13969
Northern Saw-whet Owl      10790
Short-eared Owl             9022
Long-eared Owl              8152
Northern Goshawk            2985
Broad-winged Hawk           2755
Zone-tailed Hawk            2026
Flammulated Owl             1914
Barred Owl                  1310
Name: COMMON NAME, dtype: int64


In [5]:
df['Main Colors'] = df['Main Colors'].str.replace(" ", "")
df['Main Colors'] = df['Main Colors'].str.split(",")


In [6]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
# df['Main_Colors_mlb'] = mlb.fit_transform(df['Main Colors'])
# print(mlb.fit_transform(df['Main Colors']))
# print(mlb.classes_)
df_mlb = pd.DataFrame(mlb.fit_transform(df['Main Colors']),columns=mlb.classes_)
df.dtypes
# print(df['Main Colors_mlb'].head())

Unnamed: 0.1          int64
LAST EDITED DATE     object
COMMON NAME          object
SCIENTIFIC NAME      object
COUNTY               object
COUNTY CODE          object
LATITUDE            float64
LONGITUDE           float64
OBSERVATION DATE     object
Size                 object
Main Colors          object
Brown                 int64
White                 int64
Gray                  int64
Orange                int64
Black                 int64
Bird                 object
Category             object
Hawk                  int64
Falcon                int64
Kite                  int64
Eagle                 int64
Owl                   int64
Kestrel               int64
dtype: object

In [7]:
# print(df.shape)
# print(mlb.fit_transform(df['Main Colors']).shape)
df_mlb.dtypes

black     int64
brown     int64
gray      int64
orange    int64
white     int64
dtype: object

In [5]:
#adding dummy variables for size (already did color)

df["between sparrow and robin"] = np.where(df["Size"].str.contains("between sparrow and robin"), 1, 0)
df["robin-sized"] = np.where(df["Size"].str.contains("robin-sized"), 1, 0)
df["between robin and crow"] = np.where(df["Size"].str.contains("between robin and crow"), 1, 0)
df["crow-sized"] = np.where(df["Size"].str.contains("crow-sized"), 1, 0)
df["between crow and goose"] = np.where(df["Size"].str.contains("between crow and goose"), 1, 0)
df["goose-sized or larger"] = np.where(df["Size"].str.contains("goose-sized or larger"), 1, 0)



In [6]:
# Note that Lat/Long coordinates can often 
#be used as-is with tree-based models like Random Forest or Gradient Boost that do not require data to be normalized. 
#Other models such as neural network models would usually require those coordinates to be normalized.

#Normalizing the Lat and Long columns

df['Lat_normalized'] = (df['LATITUDE'] - df['LATITUDE'].min()) / (df['LATITUDE'].max() - df['LATITUDE'].min())
df['Long_normalized'] = (df['LONGITUDE'] - df['LONGITUDE'].min()) / (df['LONGITUDE'].max() - df['LONGITUDE'].min())
df.drop(['Unnamed: 0.1', 'COUNTY CODE', 'COUNTY', 'Main Colors', 'LATITUDE', 'LONGITUDE',
         'LAST EDITED DATE'], axis=1, inplace=True)


In [7]:
df["COMMON NAME"] = df["COMMON NAME"].astype('category')
df["COMMON NAME_cat"] = df["COMMON NAME"].cat.codes
df['Size'] = df['Size'].astype('category')
df["Size_cat"] = df["Size"].cat.codes
df["Category"] = df["Category"].astype('category')
df["Birdtype_cat"] = df["Category"].cat.codes

In [8]:
print(df.columns)
# print(df[['OBSERVATION DATE', 'Brown', 'White',
#        'Gray', 'Orange', 'Black','between sparrow and robin', 'robin-sized',
#        'between robin and crow', 'crow-sized', 'between crow and goose',
#        'goose-sized or larger', 'Lat_normalized', 'Long_normalized']].head())

print(df[['OBSERVATION DATE', 'Brown', 'White',
       'Gray', 'Orange', 'Black', 'Size_cat', 'Birdtype_cat', 'Lat_normalized', 'Long_normalized']].head())

Index(['COMMON NAME', 'SCIENTIFIC NAME', 'OBSERVATION DATE', 'Size', 'Brown',
       'White', 'Gray', 'Orange', 'Black', 'Bird', 'Category', 'Hawk',
       'Falcon', 'Kite', 'Eagle', 'Owl', 'Kestrel',
       'between sparrow and robin', 'robin-sized', 'between robin and crow',
       'crow-sized', 'between crow and goose', 'goose-sized or larger',
       'Lat_normalized', 'Long_normalized', 'COMMON NAME_cat', 'Size_cat',
       'Birdtype_cat'],
      dtype='object')
  OBSERVATION DATE  Brown  White  Gray  Orange  Black  Size_cat  Birdtype_cat  \
0       1999-01-18      1      1     0       0      0         4             0   
1       1996-10-13      0      0     1       1      0         3             2   
2       1997-08-22      1      1     0       1      0         0             2   
3       1995-09-11      0      0     1       1      1         5             3   
4       1994-10-15      0      1     1       0      1         0             4   

   Lat_normalized  Long_normalized  
0    

In [9]:
save_file = "/Users/olivianystrom/Desktop/Springboard/CA_Birds_of_Prey/Birds_of_Prey_ML.csv"
df.to_csv(save_file)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
open("/Users/olivianystrom/Desktop/Springboard/CA_Birds_of_Prey/Birds_of_Prey_ML.csv")
df = pd.read_csv("/Users/olivianystrom/Desktop/Springboard/CA_Birds_of_Prey/Birds_of_Prey_ML.csv")
print(df.head())

In [ ]:
# Split into test and train data

X_train, X_test, y_train, y_test = train_test_split(df[['Brown', 'White',
       'Gray', 'Orange', 'Black','between sparrow and robin', 'robin-sized',
       'between robin and crow', 'crow-sized', 'between crow and goose',
       'goose-sized or larger', 'Lat_normalized', 'Long_normalized']], df[[
       'Hawk', 'Falcon', 'Kite', 'Eagle','Owl', 'Kestrel']], 
       test_size=0.33, random_state=42)

X_train_name, X_test_name, y_train_name, y_test_name = train_test_split(df[['Brown', 'White',
       'Gray', 'Orange', 'Black','between sparrow and robin', 'robin-sized',
       'between robin and crow', 'crow-sized', 'between crow and goose',
       'goose-sized or larger', 'Lat_normalized', 'Long_normalized']], df['COMMON NAME_cat'], 
       test_size=0.33, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,roc_auc_score